In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
import util
from model import Generator, Discriminator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers

/home/cheyu/anaconda3/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# from numba import cuda
# os.environ['CUDA_VISIBLE_DEVICES']="0"
# print(tf.test.is_gpu_available())

In [2]:
df=pd.read_csv("./dataset/df_noOutliner_ana.csv",index_col=0)
df=util.FeatureArrange(df)

In [3]:
sc = MinMaxScaler()
sc.fit(df)
df_mm = sc.transform(df)
df.shape

(4778, 73)

In [ ]:
## setting hyperparameter
latent_dim = 73
epochs = 1000
batch_size= 128
buffer_size = 6000
save_interval = 50
n_critic = 5

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
gen_opt = tf.keras.optimizers.Adam(0.0001, 0.5, 0.9)
disc_opt = tf.keras.optimizers.Adam(0.0001, 0.5, 0.9)

In [ ]:
train_dataset = train_data.shuffle(buffer_size).batch(batch_size)

In [ ]:
@tf.function
def train_discriminator(x):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as dis_tape:
        gen_data = Generator(noise,  training=True)

        gen_output = discriminator(gen_data, training=True)
        real_output = discriminator(x, training=True)

        x_hat = util.random_weight_average([x, gen_data])
        d_hat = discriminator(x_hat, training=True)

        disc_loss = util.discriminator_loss(real_output, gen_output, d_hat, x_hat)

    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))

    return disc_loss


@tf.function
def train_generator():
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape:
        gen_data = generator(noise)
        gen_output = discriminator(generated_imgs, training=True)

        gen_loss = util.generator_loss(gen_output)

    grad_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))

    return gen_loss

In [ ]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0
    for data in train_dataset:
        disc_loss += train_discriminator(data)

        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator()
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1,
                                                                               time.time() - start, gen_loss / batch_size, disc_loss / (batch_size*n_critic)))

In [10]:
# tf.random.uniform([73,1],0,1)